In [1]:
import numpy as np
import cupy as cp

cp.random.seed(415)

In [2]:
def reinforce_function(func, n, *args):
    """ Repeat a function with certain parameters several times and return the
    mean result

    Parameters
    ----------
    func : function
        The function to be repeated
    n : int
        The repeating time
    args : tuple
        The parameters for the function

    Returns
    -------
    mean_value : float
        The mean value of n times running
    """

    n = int(n)
    value = []
    for i in range(n):
        value.append(func(*args))
    value = cp.array(value)
    mean_value = cp.mean(value, axis=0)

    return mean_value


# Parameters

In [3]:
unis_stock = 0.05  # Unissued stock options' ratio
sigma = 0.9  # Volatility
lam = 0.25  # Exit rate
rf = 0.025  # Risk-free rate

# Baseline Scenario

In [4]:
def baseline_valuation(n, x0, unis_stock, sigma, lam, rf):
    # Convert the simulation size into an integer
    n = int(n)
    # Initialize parameters of the asset
    pa, pb, ia, ib = 450 * (1 - unis_stock), 1000 * (1 - unis_stock), 50, 100
    # Calculate the holding ratios for all investors
    rb = ib / pb  # Series B Investor
    ra = (1 - rb) * ia / pa  # Series A Investor
    nc = pb - ib - ib / rb * ra  # Number of Common shares
    # Generate random variables
    t = cp.random.exponential(1 / lam, size=n)
    z = cp.random.normal(size=n)
    # Calculate the exit value
    xt = x0 * cp.exp((rf - sigma ** 2 / 2) * t + sigma * cp.sqrt(t) * z)
    # Calculate the payout to the new investor
    fb_ipo = xt * rb
    fb_mna = cp.maximum(cp.minimum(ib * xt / (ia + ib), ib), xt * rb)
    # Calculate the payout to common shareholders
    fc_ipo = xt * (1 - ra - rb)
    fa_mna = cp.maximum(cp.minimum(ia * xt / (ia + ib), ia), xt * ra)
    fc_mna = xt - fb_mna - fa_mna
    # Calculate the proability of an IPO
    p_ipo = cp.ones(n)
    p_ipo[xt <= 32] = 0
    p_ipo[(xt > 32) & (xt <= 1000)] = 0.65 * (
        cp.log(xt[(xt > 32) & (xt <= 1000)] / 32)) / (cp.log(1000 / 32))
    p_ipo[(xt > 1000) & (xt <= 100000)] = 0.65 + 0.2 * (
        cp.log(xt[(xt > 1000) & (xt <= 100000)] / 1000)) / (cp.log(100))
    # Generate the outcomes
    outcome_b = p_ipo * fb_ipo + (1 - p_ipo) * fb_mna
    outcome_c = p_ipo * fc_ipo + (1 - p_ipo) * fc_mna
    eob = cp.mean(cp.exp(-rf * t) * outcome_b)
    eoec = cp.mean(cp.exp(-rf * t) * outcome_c) / nc

    return eob, eoec


In [5]:
# Output the results
x0 = 771
print("The fair investment for Series B investor: ${:.0f}m\n"
      "The fair value of common shares: ${:.2f}".format(
      *reinforce_function(baseline_valuation,
                          1e3, 1e7, x0, unis_stock, sigma, lam, rf)))

The fair investment for Series B investor: $100m
The fair value of common shares: $0.78


# Automatic Conversion Veto Scenario

In [6]:
def auto_conv_veto_valuation(n, x0, unis_stock, sigma, lam, rf, acv_rate):
    # Convert the simulation size into an integer
    n = int(n)
    # Initialize parameters of the asset
    pa, pb, ia, ib = 450 * (1 - unis_stock), 1000 * (1 - unis_stock), 50, 100
    # Calculate the holding ratios for all investors
    rb = ib / pb  # Series B Investor
    ra = (1 - rb) * ia / pa  # Series A Investor
    nc = pb - ib - ib / rb * ra  # Number of Common shares
    # Generate random variables
    t = cp.random.exponential(1 / lam, size=n)
    z = cp.random.normal(size=n)
    # Calculate the exit value
    xt = x0 * cp.exp((rf - sigma ** 2 / 2) * t + sigma * cp.sqrt(t) * z)
    # Calculate the payout to the new investor
    fb_mna = cp.maximum(cp.minimum(ib * xt / (ia + ib), ib), xt * rb)
    fb_ipo = cp.where(fb_mna > acv_rate * xt * rb, fb_mna, xt * rb)
    # Calculate the payout to common shareholders
    fa_mna = cp.maximum(cp.minimum(ia * xt / (ia + ib), ia), xt * ra)
    fa_ipo = cp.where(fa_mna > acv_rate * xt * ra, fa_mna, xt * ra)
    fc_mna = xt - fb_mna - fa_mna
    fc_ipo = xt - fb_ipo - fa_ipo
    # Calculate the proability of an IPO
    p_ipo = cp.ones(n)
    p_ipo[xt <= 32] = 0
    p_ipo[(xt > 32) & (xt <= 1000)] = 0.65 * (
        cp.log(xt[(xt > 32) & (xt <= 1000)] / 32)) / (cp.log(1000 / 32))
    p_ipo[(xt > 1000) & (xt <= 100000)] = 0.65 + 0.2 * (
        cp.log(xt[(xt > 1000) & (xt <= 100000)] / 1000)) / (cp.log(100))
    # Generate the outcomes
    outcome_b = p_ipo * fb_ipo + (1 - p_ipo) * fb_mna
    outcome_c = p_ipo * fc_ipo + (1 - p_ipo) * fc_mna
    eob = cp.mean(cp.exp(-rf * t) * outcome_b)
    eoec = cp.mean(cp.exp(-rf * t) * outcome_c) / nc

    return eob, eoec


In [7]:
# Output the results
x0 = 651
acv_rate = 0.75
print("The fair investment for Series B investor: ${:.0f}m\n"
      "The fair value of common shares: ${:.2f}".format(
      *reinforce_function(auto_conv_veto_valuation,
                          1e3, 1e7, x0, unis_stock, sigma, lam, rf, acv_rate)))

The fair investment for Series B investor: $100m
The fair value of common shares: $0.63
